# 합성곱 신경망

합성곱 신경망에 대한 전반적인 내용을 MNIST 숫자 분류 예제를 통해 알아보자.
이전의 CNN을 사용하면 DNN을 사용한 MNIST 분류보다 정확도를 앞지를 것이다.(Convolutional Neural Network, Convnet은 동일한 의미)

convnet은 기본적으로 Conv2D와 MaxPooling2D층을 쌓아서 구성한다.
첫번째 인자는 채널의 갯수, 두번째 인자는 윈도우의 크기, 사용할 세번째 인자는 활성화 함수의 종류, 마지막 인자는 입력으로 사용할 텐서의 크기이다.  
여기서는 마지막 인자로 MNIST 이미지 포맷인 (28, 28, 1) 크기의 입력을 처리하도록 컨브넷을 설정해야 한다.  
MaxPooling2D()에서 (2,2)크기의 윈도우로 downsampling시키는데 다운 샘플링을 통해서 더 정확한 패턴을 인식할 수 있다고 한다. 그 이유는 다음에 보도록 하자.  

In [16]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


위에서 구성한 Conv2D와 MaxPooling2D 층들의 출력은 (height, width, channels) 크기의 3D 텐서이다.  
높이와 넓이 차원은 네트워크가 깊어질수록 작아지고  
채널의 수는 Conv2D 층에 전달된 첫 번째 매개변수에 따라 32개 또는 64개로 출력되게 된다.  

다음 단계로, 위에서 생성된 마지막 층의 ((3, 3, 64) 크기인) 출력 텐서를 완전 연결 네트워크에 주입하자.  
이 완전 연결 네트워크는 이미 익숙하게 보았던 Dense 층을 쌓은 분류기인데,  
이 분류기는 1D 벡터를 사용하는데 이전 층의 출력이 3D 텐서이므로 3D 출력을 1D 텐서로 변환한 다음, 몇 개의 Dense 층을 추가한다.  
10개의 클래스를 분류하기 위해 마지막 층의 출력 크기를 10으로 하고 활성화 함수로는 소프트맥스를 사용하자. 구성된 전체 네트워크는 다음과 같다.  

In [17]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_5 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                36928     
__________

실행 결과를 살펴보면,  
(3, 3, 64) 출력이 (576,) 크기의 벡터로 펼쳐진 후 Dense 층으로 주입된 것을 볼 수 있다.

MNIST 숫자 이미지에 이 컨브넷을 훈련시켜보자. 이전의 MNIST 예제를 재활용..

In [18]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape(60000, 28, 28, 1)
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [20]:
model.compile(optimizer='rmsprop',
            loss='categorical_crossentropy',
            metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 43s 714us/step - loss: 0.1812 - acc: 0.9423
Epoch 2/5
60000/60000 [==============================] - 42s 705us/step - loss: 0.0491 - acc: 0.9847
Epoch 3/5
60000/60000 [==============================] - 43s 710us/step - loss: 0.0346 - acc: 0.9894
Epoch 4/5
60000/60000 [==============================] - 45s 751us/step - loss: 0.0269 - acc: 0.9920
Epoch 5/5
60000/60000 [==============================] - 44s 741us/step - loss: 0.0220 - acc: 0.9931


이제 모델을 평가해보자.


In [21]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 3s 256us/step


In [23]:
print(test_acc)
print(test_loss)

0.9903
0.027772899679182592


약 99%로 이전(94~95%)보다 훨씬 높은 정확도를 보이고 있다.